In [ ]:
import pandas as pd
import re
import numpy as np
import os

info_full = pd.read_csv('/Users/dinhvan/Projects/Administrative Units/administrative_units.csv',dtype = str)
province = pd.read_csv('/Users/dinhvan/Projects/Administrative Units/province.csv',dtype = str)

ma_ttp = dict(zip(province['province_name'],province['province_code']))
ma_mien = dict(zip(province['province_code'], province['region_code']))
list_ttp = province['province_name'].values.tolist()
list_ttp = list_ttp[:-1]

Phone_Prefix_Vina = {
    "096": "Viettel",
    "097": "Viettel",
    "098": "Viettel",
    "032": "Viettel",
    "033": "Viettel",
    "034": "Viettel",
    "035": "Viettel",
    "036": "Viettel",
    "037": "Viettel",
    "038": "Viettel",
    "039": "Viettel",
    "086": "Viettel",
    "090": "Mobifone",
    "093": "Mobifone",
    "070": "Mobifone",
    "089": "Mobifone",
    "077": "Mobifone",
    "076": "Mobifone",
    "078": "Mobifone",
    "079": "Mobifone",
    "091": "Vinaphone",
    "094": "Vinaphone",
    "081": "Vinaphone",
    "082": "Vinaphone",
    "083": "Vinaphone",
    "084": "Vinaphone",
    "085": "Vinaphone",
    "088": "Vinaphone",
    "099": "Gmobile",
    "059": "Gmobile",
    "092": "Vietnamobile",
    "056": "Vietnamobile",
    "058": "Vietnamobile",
}

def get_correct_email(email):
    if email is None or email == '':
        return None
    else:
        match = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', email)
        if len(match) == 0:
            return None
        else:
            return match[0]

def get_list_solved(list_ex):
    content_dict = dict.fromkeys(['address', 'phone', 'fax', 'email'])
    for vl in list_ex:
        if 'Address:' in vl:
            content_dict['address'] = vl
        if ('Phone:' in vl) or ('Tel' in vl):
            content_dict['phone'] = vl
        if 'Fax:' in vl:
            content_dict['fax'] = vl
        if 'Email:' in vl:
            content_dict['email'] = vl
    return content_dict

def handle_phone_number(phone_number):
    special_characters = ['(',')','.',' ','+','Work','Mobile']
    for ch in special_characters:
        if ch in phone_number:
            phone_number = phone_number.replace(ch, '')
            
    phone_number = phone_number.replace('hoặc', '-')
    phone_number = phone_number.replace('Hoặc', '-')
    phone_number = phone_number.replace(',', '-')
    phone_number = phone_number.replace('&', '-')
    
    if phone_number == '' or phone_number is None:
        return None
    else:
        if 'E' in phone_number:
            return None
        elif phone_number[:3] == '840':
            phone_number = phone_number[2:]
        elif phone_number[:2] == '84':
            phone_number = '0' + phone_number[2:]
        
        if len(phone_number) < 8 or (len(phone_number) == 8  and phone_number[0] != '1'):
            return None
        elif len(phone_number) == 9 and (phone_number[0] == '0' or phone_number[0] == '1'):
            return None
        elif len(phone_number) == 10 and phone_number[0] != '2' and phone_number[:1] != '02' and phone_number[0] != '0':
            return None
        elif len(phone_number) == 11 and phone_number[0] != '0':
            return None
            
        if (phone_number[0] != '0' and phone_number[0] != '1') and phone_number is not None:
            phone_number = '0' + phone_number

        return phone_number
 
def no_accent_vietnamese(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

def get_location_name(s):
    s = re.sub(r'^thành phố|^huyện|^quận|^thị xã|^tx|^tt|^xã|^thị trấn|^tp|^tỉnh|^phường', '', s)
    s = s.strip()
    return s

def get_city_name(location, list_ttp):
    location = no_accent_vietnamese(location)
    location = location.lower()
    location = location.replace(' ','')
    
    list_ttp_solved = [get_location_name(city.lower()) for city in list_ttp]
    list_ttp_solved = [no_accent_vietnamese(city_lower) for city_lower in list_ttp_solved]
    list_ttp_solved = [city_lower.replace(' ','') for city_lower in list_ttp_solved]
    
    list_index = [location.rfind(city) for city in list_ttp_solved ]
    max_value = max(list_index)
    if max_value == -1:
        return ''
    else:
        index_max_value = list_index.index(max_value)
        return list_ttp[index_max_value]

def get_city_code(city, ma_ttp):
    return ma_ttp[city]

def get_city_area(city_code, ma_mien):
    return ma_mien[city_code]

def get_district_town_name(city,location,info_full):
    town, district = '',''
    
    full_district = info_full.loc[info_full['Tỉnh Thành Phố'] == city]
    full_district = full_district.drop_duplicates(subset=['Quận Huyện'], keep='first', ignore_index=True)
    
    city_solved = get_location_name(city.lower())
    city_solved = no_accent_vietnamese(city_solved).replace(' ','')
    
    list_district = full_district['Quận Huyện'].values.tolist()
    list_district.sort(key=lambda s: len(s))
    list_district_lower = [get_location_name(district.lower()) for district in list_district]
    list_district_lower = [no_accent_vietnamese(district) for district in list_district_lower]
    list_district_lower = [district.replace(' ','') for district in list_district_lower]
        
    location = location.lower()
    location = location.replace(' ','')
    # index_city = location.rfind(city_solved)
    location = no_accent_vietnamese(location)
    index_city = location.rfind(city_solved)
    # location = location.replace(' ','')
    
    if index_city == -1:
        return ['','']
    else: 
        location = location[:index_city]
    
        list_index_district = [location.rfind(value) for value in list_district_lower]
        max_value_district = -1
        if len(list_index_district) <= 1:
            return ['','']
        else:
            for i in reversed(list_index_district):
                if i != -1:
                    max_value_district = i
                    break
            if max_value_district == -1:
                return ['','']
            else:
                index_max_value_district = [index for index, item in enumerate(list_index_district) if item == max_value_district][-1]
                district = list_district[index_max_value_district]

                full_town = info_full.loc[(info_full['Tỉnh Thành Phố'] == city) & (info_full['Quận Huyện'] == district)]
                
                list_town = full_town['Phường Xã'].values.tolist()
                list_town.sort(key=lambda s: len(s))
                list_town_solved = [get_location_name(town.lower()) for town in list_town]
                list_town_solved = [no_accent_vietnamese(town) for town in list_town_solved]
                list_town_solved = [town.replace(' ','') for town in list_town_solved]
                
                district_solve = get_location_name(district.lower())
                district_solve = no_accent_vietnamese(district_solve)
                district_solve = district_solve.replace(' ','')
                
                index_district = location.rfind(district_solve)
                location = location[:index_district]
                
                list_index_town= [location.rfind(value) for value in list_town_solved]
                max_value_town = -1
                if len(list_index_town) <= 1:
                    return [district,'']
                else:
                    for i in reversed(list_index_town):
                        if i != -1:
                            max_value_town = i
                            break
                    if max_value_town == -1:
                        return [district,'']
                    else:
                        index_max_value_town = [index for index, item in enumerate(list_index_town) if item == max_value_town][-1]
                        town = list_town[index_max_value_town]
                        return [district,town]

def get_type(url_source):
    type_source = ''
    if 'spa-massage-c21.html' in  url_source:
        type_source = 'Spa/Massage'
    if 'the-duc-tham-my-c22.html' in url_source:
        type_source = 'Thể dục thẩm mỹ'
    if 'cham-soc-da-c24.html' in url_source:
        type_source = 'Chăm sóc da'
    if 'shop-hoa-my-pham-c25.html' in url_source:
        type_source = 'Shop hóa mỹ phẩm' 
    if 'phong-kham-c30.html' in url_source:
        type_source = 'Phòng khám'
    if 'nha-thuoc-c32.html' in url_source:
        type_source = 'Nhà thuốc'
    if 'resort-c49.html' in url_source:
        type_source = 'Resort'
    return type_source      
